In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('DF').getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.types import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6bcb46f3997c6db07f9d777badfef651f2b25109ad21f1f454361ba403bd4b21
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
accidents_df =spark.read.csv('/content/Road_accidents.csv',header=True,inferSchema=True)
accidents_df.show()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        8|     14|       6|      18|
| A & N Islands|200

In [3]:
accidents_time_df =spark.read.csv('/content/Road_accidents_time.csv',header=True,inferSchema=True)
accidents_time_df.show()

+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|      STATE/UT|YEAR|0-3 hrs. (Night)|3-6 hrs. (Night)|6-9 hrs (Day)|9-12 hrs (Day)|12-15 hrs (Day)|15-18 hrs (Day)|18-21 hrs (Night)|21-24 hrs (Night)|
+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
| A & N Islands|2001|               2|               6|           29|            40|             39|             40|               18|                7|
| A & N Islands|2002|               2|               6|           22|            41|             33|             33|               23|                8|
| A & N Islands|2003|               2|               8|           31|            35|             28|             36|               25|               15|
| A & N Islands|2004|               2|               5|           29|            4

In [4]:
from pyspark.sql.window import Window

In [5]:
months = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']
total_expr = " + ".join([f"`{month}`" for month in months])
accidents_df = accidents_df.withColumn("Total", expr(total_expr).cast("int"))
accidents_df.show()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|Total|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|  181|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|  168|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|  180|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  215|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  206|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        

In [6]:
accidents_df.select('STATE/UT','Total').filter(col('Total')==accidents_df.agg({'Total':'max'}).collect()[0][0]).show()

+----------+-----+
|  STATE/UT|Total|
+----------+-----+
|Tamil Nadu|75480|
+----------+-----+



In [7]:
accidents_df.groupBy('STATE/UT').agg({'Total':'avg'}).show()

+-----------------+------------------+
|         STATE/UT|        avg(Total)|
+-----------------+------------------+
|         Nagaland|55.785714285714285|
|        Karnataka|41768.642857142855|
|           Odisha| 8231.214285714286|
|           Kerala|           37011.5|
|       Tamil Nadu|60862.357142857145|
|     Chhattisgarh|            8415.5|
|   Andhra Pradesh|39058.642857142855|
|      Lakshadweep|               1.0|
|   Madhya Pradesh| 28143.35714285714|
|           Punjab|3343.1428571428573|
|          Manipur| 580.7142857142857|
|      Daman & Diu| 45.92857142857143|
|  Jammu & Kashmir| 5915.714285714285|
|              Goa|3209.3571428571427|
|          Mizoram| 79.71428571428571|
| Himachal Pradesh|2650.3571428571427|
|       Puducherry|            1513.5|
|          Haryana|  9422.92857142857|
|        Jharkhand|3300.6428571428573|
|Arunachal Pradesh|242.07142857142858|
+-----------------+------------------+
only showing top 20 rows



In [8]:
accidents_df.groupBy('STATE/UT').agg({'JANUARY':'avg','FEBRUARY':'avg','MARCH':'avg','APRIL':'avg','MAY':'avg','JUNE':'avg','JULY':'avg','AUGUST':'avg','SEPTEMBER':'avg','OCTOBER':'avg','NOVEMBER':'avg','DECEMBER':'avg'}).show()

+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+
|         STATE/UT|     avg(DECEMBER)|         avg(APRIL)|          avg(JULY)|     avg(SEPTEMBER)|         avg(JUNE)|      avg(FEBRUARY)|       avg(OCTOBER)|       avg(AUGUST)|      avg(NOVEMBER)|        avg(MARCH)|          avg(MAY)|      avg(JANUARY)|
+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+
|         Nagaland| 5.142857142857143|  4.928571428571429|  5.071428571428571| 3.0714285714285716| 5.357142857142857|  5.857142857142857|  5.214285714285714| 3.642857142857143| 3.2857142857142856| 5.142857142857143|               4.0| 5.0

In [9]:
accidents_df.groupBy('STATE/UT').agg({'JANUARY':'max','FEBRUARY':'max','MARCH':'max','APRIL':'max','MAY':'max','JUNE':'max','JULY':'max','AUGUST':'max','SEPTEMBER':'max','OCTOBER':'max','NOVEMBER':'max','DECEMBER':'max'}).show()

+-----------------+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|         STATE/UT|max(DECEMBER)|max(APRIL)|max(JULY)|max(SEPTEMBER)|max(JUNE)|max(FEBRUARY)|max(OCTOBER)|max(AUGUST)|max(NOVEMBER)|max(MARCH)|max(MAY)|max(JANUARY)|
+-----------------+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|         Nagaland|           14|         9|       16|            10|       13|           10|          10|          6|           12|        12|      10|          10|
|        Karnataka|         4170|      4295|     3648|          3527|     3942|         3972|        3830|       3619|         3787|      4212|    4538|        4092|
|           Odisha|          931|       794|      763|           707|      846|          826|         776|        719|          854|       912|     999|         911|
|   

In [10]:
high=accidents_df.groupBy('STATE/UT').agg({'JANUARY':'sum','FEBRUARY':'sum','MARCH':'sum','APRIL':'sum','MAY':'sum','JUNE':'sum','JULY':'sum','AUGUST':'sum','SEPTEMBER':'sum','OCTOBER':'sum','NOVEMBER':'sum','DECEMBER':'sum'})

In [11]:
high.orderBy('STATE/UT').show()

+-----------------+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|         STATE/UT|sum(DECEMBER)|sum(APRIL)|sum(JULY)|sum(SEPTEMBER)|sum(JUNE)|sum(FEBRUARY)|sum(OCTOBER)|sum(AUGUST)|sum(NOVEMBER)|sum(MARCH)|sum(MAY)|sum(JANUARY)|
+-----------------+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|    A & N Islands|          269|       256|      217|           238|      238|          246|         267|        234|          205|       248|     211|         264|
|   Andhra Pradesh|        47524|     46893|    42374|         41520|    46570|        44657|       43697|      43947|        44013|     47732|   51442|       46452|
|Arunachal Pradesh|          316|       253|      238|           284|      263|          298|         290|        247|          301|       295|     298|         306|
|   

In [12]:
accident_month=accidents_df.agg({'JANUARY':'sum','FEBRUARY':'sum','MARCH':'sum','APRIL':'sum','MAY':'sum','JUNE':'sum','JULY':'sum','AUGUST':'sum','SEPTEMBER':'sum','OCTOBER':'sum','NOVEMBER':'sum','DECEMBER':'sum'})

In [13]:
accident_month.show()

+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|sum(DECEMBER)|sum(APRIL)|sum(JULY)|sum(SEPTEMBER)|sum(JUNE)|sum(FEBRUARY)|sum(OCTOBER)|sum(AUGUST)|sum(NOVEMBER)|sum(MARCH)|sum(MAY)|sum(JANUARY)|
+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+
|       473905|    479663|   440263|        435302|   473574|       459272|      454961|     438351|       457192|    486141|  521563|      482719|
+-------------+----------+---------+--------------+---------+-------------+------------+-----------+-------------+----------+--------+------------+



In [14]:
df_max_month=accident_month.toPandas()

In [15]:
lst=df_max_month.values

In [16]:
lst.max()

521563

In [20]:

#Which month has more accidents over the year?
highest_acc = accidents_df.select([col for col in accidents_df.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+-----+
|month|value|
+-----+-----+
|Total|75480|
+-----+-----+

